In [5]:
import os
import re
import html
import urllib.request

def get_html(page_url):
    """Загружает html-текст по заданной ссылке
    и обрабатывает исключения.
    """
    try:
        page = urllib.request.urlopen(page_url)
        return page.read().decode('utf-8')
    except Exception:
        print('Error at', page_url)
        return

In [6]:
def find_urls(html_content):
    """Находит в html-тексте все ссылки."""
    return re.findall('<a href="(.*?)"', html_content, flags=re.DOTALL)

In [7]:
def find_info(html_content):
    """Находит в html-тексте информацию об
    авторе, заголовке, дате публикации и категории статьи.
    """
    author = re.findall(
        '<h3 class="author-title author">'
        '<a href=".*?" title=".*?" rel="author">(.*?)</a>',
        html_content, flags=re.DOTALL)[0]
    if author == '':
        author = None
    header = re.findall(
        '<h1 class="entry-title">(.*?)</h1>',
        html_content, flags=re.DOTALL)[0]
    date = re.findall(
        '<time class="entry-date updated published" '
        'datetime=".*?">(.*?)</time>',
        html_content, flags=re.DOTALL)[0]
    topic = re.findall(
        '<span class="entry-category">'
        '<a href=".*?" rel="category tag">(.*?)</a>',
        html_content, flags=re.DOTALL)[0]
    return author, header, date, topic

In [8]:
def plain_text(html_content):
    """Очищает html-текст от тэгов, приводит в режим чтения,
    обрабатывает специальные символы.
    """
    text = re.findall(
        '<div class="entry-content">(.*?)</div>',
        html_content, flags=re.DOTALL)[0]
    text = re.sub('<script>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub('<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub('<.*?>', '', text, flags=re.DOTALL)
    return html.unescape(text)

In [9]:
def catalogue(s, date):
    """Создаёт каталог из папок."""
    year = date.split('.')[2]
    directory = '.\\Tamlife\\%s\\%s\\%s' % (s, year, date)
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

In [10]:
def mystem(page_id, input_file, output_file1, output_file2):
    """Создаёт xml- и txt-файлы, размеченные с помощью mystem, из данного."""
    os.system(''.join([
        'C:\\Users\\ersho\\jupyter_github\\mystem.exe '
        '-c -l -i -d --eng-gr --format xml ',
        input_file, os.sep, page_id, '.txt ',
        output_file1, os.sep, page_id, '.xml']))
    os.system(''.join([
        'C:\\Users\\ersho\\jupyter_github\\mystem.exe '
        '-c -l -i -d --eng-gr ',
        input_file, os.sep, page_id, '.txt ',
        output_file2, os.sep, page_id, '.txt']))

In [11]:
def update_csv_file(path, author, header, date, topic, url):
    """Добавляет в csv-файл информацию о статье."""
    with open('.\\Tamlife\\metadata.csv', 'a', encoding='utf-8') as f:
        year = date.split('.')[2]
        f.write(
            '%s\t%s\t%s\t%s\tпублицистика\t'
            '%s\tнейтральный\tн-возраст\tн-уровень\tрайонная\t'
            '%s\tТамбовская жизнь\t'
            '%s\tгазета\tРоссия\tТамбовская область\tru\n'
            % (path, author, header, date, topic, url, year))

In [12]:
def main():
    page_url_base = 'http://tamlife.ru/news'
    for page_number in range(1, 194):
        page_url = page_url_base
        if page_number > 1:
            page_url += '/page/%s' % page_number
        urls = find_urls(get_html(page_url))
        for url in urls:
            if re.fullmatch(
                    r'http://tamlife\.ru/news/[A-Za-z]+?/[0-9]+?\.html',
                    url) is not None:
                html = get_html(url)
                author, header, date, topic = find_info(html)
                page_id = re.findall(
                    r'.*?/%s([0-9]+?)\.html' % ''.join(
                        reversed(date.split('.'))),
                    url)[0]
                text = plain_text(html)
                path = catalogue('plain', date)
                text_file = ''.join([path, os.sep, page_id, '.txt'])
                if not os.path.isfile(text_file):
                    with open(text_file, 'w', encoding='utf-8') as f:
                        f.write(text)
                    path_mystem_xml = catalogue('mystem_xml', date)
                    path_mystem_plain = catalogue('mystem_plain', date)
                    mystem(page_id, path, path_mystem_xml, path_mystem_plain)
                    with open(text_file, 'w', encoding='utf-8') as f:
                        f.write(
                            '@au %s\n@ti %s\n@da %s\n@topic %s\n@url %s\n%s'
                            % (author, header, date, topic, url, text))
                    update_csv_file(
                        ''.join([
                            path.lstrip('.\\'),
                            os.sep, page_id, '.txt ']),
                        author, header, date, topic, url)

In [13]:
if __name__ == '__main__':
    main()

Error at http://tamlife.ru/news/sport/2018102612423913128.html


TypeError: expected string or bytes-like object